In [20]:
#import libraries
import numpy as np
import pandas as pd
import pickle
import datetime

#show max rows in pandas
pd.set_option('display.max_rows', 500)

In [15]:
full_pop = pd.read_json('demo_data.json')

In [12]:
full_pop.head()

,location_id,time,temperature_2m,relative_humidity_2m,rain,pressure_msl,surface_pressure,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,soil_temperature_0_to_7cm,county_name,wind_shear
0,0,2024-05-21T00:00:00,61.7,88,0.272,1007.7,958.3,15.6,25.5,35,35,64.4,Lyon,9.900000
1,0,2024-05-21T01:00:00,61.2,89,0.008,1006.6,957.2,13.1,22.1,37,39,63.1,Lyon,9.019574
2,0,2024-05-21T02:00:00,60.4,91,0.000,1005.6,956.2,13.1,22.8,57,60,62.2,Lyon,9.742108
3,0,2024-05-21T03:00:00,60.1,93,0.000,1005.0,955.6,11.1,19.6,54,59,61.7,Lyon,8.596846
4,0,2024-05-21T04:00:00,59.8,95,0.000,1004.0,954.6,10.8,19.1,68,72,61.4,Lyon,8.360321


In [51]:
#filter full_pop to include one timestamp
batch = full_pop[full_pop['time'] == '2024-05-21T19:00:00']

In [45]:
batch.head()

,location_id,time,temperature_2m,relative_humidity_2m,rain,pressure_msl,surface_pressure,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,soil_temperature_0_to_7cm,county_name,wind_shear
18,0,2024-05-21T18:00:00,56.2,67,0.000,998.9,949.4,26.3,38.5,322,323,61.2,Lyon,12.212634
42,1,2024-05-21T18:00:00,55.6,80,0.012,996.8,941.8,24.7,34.5,314,315,61.2,Osceola,9.813235
66,2,2024-05-21T18:00:00,60.8,83,0.008,994.0,945.9,23.4,33.1,317,318,64.6,Dickinson,9.712154
90,3,2024-05-21T18:00:00,66.5,82,0.012,992.2,947.6,13.8,19.1,306,307,66.4,Emmet,5.307569
114,4,2024-05-21T18:00:00,72.6,86,0.260,994.7,960.3,12.8,21.4,199,200,70.7,Allamakee,8.604850


In [52]:
# Assuming full_pop is your DataFrame with the desired columns
columns = full_pop.columns

# Initialize a blank DataFrame with the same columns as full_pop
batch_df = pd.DataFrame(columns=columns)

# Loop through each row in the batch and populate batch_df
for index, row in batch.iterrows():
    county_name = row['county_name']
    
    # Check if the county is already populated in batch_df
    if county_name in batch_df['county_name'].values:
        # Remove the existing record for the county
        batch_df = batch_df[batch_df['county_name'] != county_name]
    
    # Append the new row
    batch_df = pd.concat([batch_df, pd.DataFrame([row])], ignore_index=True)

C:\Users\goodm\AppData\Local\Temp\ipykernel_23744\1275678040.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  batch_df = pd.concat([batch_df, pd.DataFrame([row])], ignore_index=True)


In [53]:
batch_df.head()

,location_id,time,temperature_2m,relative_humidity_2m,rain,pressure_msl,surface_pressure,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,soil_temperature_0_to_7cm,county_name,wind_shear
0,0,2024-05-21T19:00:00,55.5,69,0.000,1000.8,951.2,24.7,35.7,318,319,60.9,Lyon,11.012202
1,1,2024-05-21T19:00:00,54.0,72,0.000,999.0,943.7,23.8,35.3,311,313,59.7,Osceola,11.544418
2,2,2024-05-21T19:00:00,55.0,82,0.008,997.1,948.3,23.6,32.9,303,304,62.1,Dickinson,9.312707
3,3,2024-05-21T19:00:00,57.7,82,0.008,995.0,949.6,24.9,34.8,300,301,64.3,Emmet,9.913322
4,4,2024-05-21T19:00:00,68.3,93,0.469,994.4,959.7,9.5,17.1,188,197,68.7,Allamakee,7.858757


In [32]:
def predict(df, model, features, max_time):
    # Initialize an empty list for storing predictions
    predictions_list = []

    # For each unique county, get the latest information and make a prediction
    for county in df['county_name'].unique():
        # Get the latest data for the current county
        latest_data = df[df['county_name'] == county].sort_values(by='time').iloc[-1]

        # Extract features for prediction
        X = latest_data[features].values.reshape(1, -1)

        # Make a prediction (probability of positive class)
        pred = model.predict_proba(X)[0][1]

        # Append the result to the list
        predictions_list.append({'Time': max_time, 'County': county, 'Prediction': pred})

    # Convert the list to a DataFrame
    df_st = pd.DataFrame(predictions_list)
    #order this by pred desc
    df_st = df_st.sort_values(by='Prediction', ascending=False)
    return df_st

In [55]:
full_pop.columns

Index(['location_id', 'time', 'temperature_2m', 'relative_humidity_2m', 'rain',
       'pressure_msl', 'surface_pressure', 'wind_speed_10m', 'wind_speed_100m',
       'wind_direction_10m', 'wind_direction_100m',
       'soil_temperature_0_to_7cm', 'county_name', 'wind_shear'],
      dtype='object')

In [44]:
df=batch
model= pickle.load(open('xgb_model.pkl', 'rb'))

features=['temperature_2m', 'relative_humidity_2m',
       'rain', 'pressure_msl', 'surface_pressure', 'wind_speed_10m',
       'wind_speed_100m', 'wind_direction_10m', 'wind_direction_100m',
       'soil_temperature_0_to_7cm', 'wind_shear']

max_time=batch['time'].max()

df_st=predict(df,model,features,max_time)
df_st.head()

,Time,County,Prediction
30,2024-05-21T18:00:00,Dubuque,0.770981
43,2024-05-21T18:00:00,Jones,0.588790
42,2024-05-21T18:00:00,Jackson,0.576660
62,2024-05-21T18:00:00,Iowa,0.570778
31,2024-05-21T18:00:00,Delaware,0.483193


In [ ]:
#streamlit
